In [20]:
import cv2
import numpy as np
import random
import tkinter as tk
from tkinter import filedialog

def select_video_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
    return file_path

def find_anomalous_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return
    
    # Initialize background subtractor
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    
    # Initialize variables for object tracking
    object_tracker = {}
    frame_count = 0
    anomalous_frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply background subtraction
        fgmask = bg_subtractor.apply(gray)
        
        # Apply morphological operations to remove noise
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, np.ones((20,20),np.uint8))
        
        # Find contours
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        current_centroids = []
        
        # Draw bounding boxes and calculate centroids
        for contour in contours:
            if cv2.contourArea(contour) < 500:  # filter out small contours
                continue
            x, y, w, h = cv2.boundingRect(contour)
            centroid = (x + w // 2, y + h // 2)
            current_centroids.append(centroid)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, centroid, 5, (0, 0, 255), -1)
        
        # Update the tracker
        for centroid in current_centroids:
            found = False
            for obj_id, positions in object_tracker.items():
                if len(positions) > 0:
                    prev_position = positions[-1]
                    if np.linalg.norm(np.array(centroid) - np.array(prev_position)) < 50:
                        object_tracker[obj_id].append(centroid)
                        found = True
                        break
            if not found:
                new_id = len(object_tracker) + 1
                object_tracker[new_id] = [centroid]
        
        # Check for anomalies in the tracked objects
        if len(object_tracker) > 1:
            directions = []
            for obj_id, positions in object_tracker.items():
                if len(positions) > 1:
                    prev_position = positions[-2]
                    curr_position = positions[-1]
                    dx = curr_position[0] - prev_position[0]
                    dy = curr_position[1] - prev_position[1]
                    direction = np.arctan2(dy, dx)
                    directions.append(direction)
            
            avg_direction = np.mean(directions)
            
            for obj_id, positions in object_tracker.items():
                if len(positions) > 1:
                    prev_position = positions[-2]
                    curr_position = positions[-1]
                    dx = curr_position[0] - prev_position[0]
                    dy = curr_position[1] - prev_position[1]
                    direction = np.arctan2(dy, dx)
                    if np.abs(direction - avg_direction) > np.pi / 2:  # Check if direction is significantly different
                        anomalous_frames.append((frame_count, frame))
                        break

        # Display frame
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to stop
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    if anomalous_frames:
        # Select a random anomalous frame to display and save
        frame_count, random_frame = random.choice(anomalous_frames)
        cv2.imshow("Random Anomalous Frame", random_frame)
        cv2.imwrite("anomaly_frame.jpg", random_frame)
        print(f"Random anomalous frame at frame count {frame_count} saved as 'anomaly_frame.jpg'")
        cv2.waitKey(0)
    else:
        print("No anomalous frames found")

if __name__ == "__main__":
    video_path = select_video_file()
    if video_path:
        find_anomalous_frame(video_path)
    else:
        print("No video file selected")


Random anomalous frame at frame count 245 saved as 'anomaly_frame.jpg'
